# Galaxy cluster mass-richness relation in DC2 extragalactic catalog

This notebook studies the mass-richness relation of galaxy clusters in DC2 extragalatic catalog as a validation test. 

Thank Nan Li for his code for analyzing clusters and searching galaxies in DC2 extragalactic catalog. 

Thank Yao-Yuan Mao for his code for fast-searching galaxies in DC2 extragalactic catalog.

Reference: 

https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/extragalactic_gcr_cluster_members.ipynb

https://github.com/LSSTDESC/gcr-catalogs/blob/master/GCRCatalogs/SCHEMA.md


Kernel: desc-python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.table import Table
from astropy.cosmology import FlatwCDM
from astropy import units as u

import GCRCatalogs
from GCR import GCRQuery
import healpy as hp

# The cosmology is related to clusters' r200 (the searching range for members) and angular diameter distances
# These parameters can be changed
cosmo = FlatwCDM(H0=71, Om0=0.264, Ob0=0.045, w0=-1.000000)


In [ ]:
# By Yao-Yuan Mao: speed up query (based on healpix)

def get_healpix_list(ra, dec, radius): 
    '''
        Find the healpix where the object is in to speed up searching and loading the object
        All values in degree
    '''
    # 32 is cosmoDC2’s healpix resolution
    return hp.query_disc(32, hp.ang2vec(ra, dec, lonlat=True), np.deg2rad(radius), inclusive=True)


def build_query_from_healpix_list(healpix_list):
    return GCRQuery((lambda x: np.in1d(x, healpix_list), 'healpix_pixel'))


def build_query_from_disc(ra, dec, radius):
    return build_query_from_healpix_list(get_healpix_list(ra, dec, radius))



In [ ]:
def rho_crit(z):
    '''
        Get the critical density at redshift z
        Input: redshift z
        Output: critical density [unit: M_sun Mpc^-3]
    '''
    # The original unit in astropy is g/cm^3
    return cosmo.critical_density(z).to(u.solMass/u.Mpc/u.Mpc/u.Mpc).value


def r200_M200(m200, z):
    '''
        Get the r200 (Mpc) of the cluster based on its m200 and z
        Input: 
                m200 [unit: M_sun]
                redshift z
        Output: r200 [unit: Mpc]
    '''
    return (3.0/4.0*m200/(200.0*rho_crit(z))/np.pi)**(1.0/3.0)


def r200_deg(m200, z):
    '''
        Input: 
                m200 [unit: M_sun]
                redshift z
        Output: r200 [unit: degree]
    '''
    D_A = cosmo.angular_diameter_distance(z).value
    
    return r200_M200(m200, z)/D_A/np.pi*180.
    
    

In [ ]:
%%time
# Now we use cosmoDC2_v1.1.4_small
# In the future we will use cosmoDC2_v1.1.4_image
galaxy_catalog = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small')


In [ ]:
# For saving the catalogs of clusters
cluster_catalog_file = "./catalog_20191124.hdf5"

In [ ]:
%%time
# Get clusters (the search is based on galaxies) 
# The filter can be changed

cluster_filters=[
                'is_central', 
                'halo_mass > 1e14', 
                'redshift_true < 0.4', 
                'redshift_true > 0.3',
                ]

cluster_data = galaxy_catalog.get_quantities([
                                      'ra_true', # Unlensed coordinate of the galaxy
                                      'dec_true', 
                                      'halo_mass', # Halo mass of the main halo that contains the galaxy
                                      'halo_id', # Unique ID of the main halo that contains the galaxy
                                      'redshift_true',
                                                 ], 
                                     filters=cluster_filters) 
                                    
cluster_table = Table(cluster_data)
print(len(cluster_table))
#print(cluster_table)
#cluster_table.write(cluster_catalog_file, 
#                    path='/cluster_table', 
#                    append=True, overwrite=True)

In [ ]:
%%time

cluster_mass = []
cluster_richness = []

for cluster_id in range(len(cluster_table)):

    halo_id = cluster_table['halo_id'][cluster_id]
    ra_cls  = cluster_table['ra_true'][cluster_id]
    dec_cls = cluster_table['dec_true'][cluster_id]
    halo_mass_cls = cluster_table['halo_mass'][cluster_id]
    redshift_cls = cluster_table['redshift_true'][cluster_id]

    # In fact, the code gives the same result without the factor 2.
    radius_cls = 2.*r200_deg(halo_mass_cls, redshift_cls) 
    #box_size_deg = 2.*radius_cls
    
    print("cluster_id: %d; halo_mass: %.2e Msun; r200: %.2e deg"%(cluster_id,
                                                                  halo_mass_cls, 
                                                                  radius_cls,
                                                                 )
         )

    # Get galaxies
    quantity_cluster_galaxy = [
#                  'galaxyID', 
                  'ra_true',
                  'dec_true',
                  'mag_true_r_lsst',   # Unlensed apparent magnitude
                  'mag_true_i_lsst',
                  'Mag_true_r_lsst_z0',
                  'Mag_true_i_lsst_z0',  # Rest-frame absolute magnitude (unlensed)
#                  'redshift_true', 
#                  'halo_mass', 
#                  'is_central',  # Whether it is BCG 
                  'halo_id',
                ]

    # The searching box can be skipped since there's a "query_from_disc" (results are the same)
    # Adding the box would increase the searching time
    filter_cluster_galaxy = [
#                     'mag_true_r_lsst < 27.', # default 30 
#                    'ra_true  >=%f'%(ra_cls-box_size_deg/2.0/np.cos(dec_cls/180.*np.pi)), 
#                    'ra_true  < %f'%(ra_cls+box_size_deg/2.0/np.cos(dec_cls/180.*np.pi)), 
#                    'dec_true >=%f'%(dec_cls-box_size_deg/2.0), 
#                    'dec_true < %f'%(dec_cls+box_size_deg/2.0),
#                     'redshift_true < 2.' (DC2 zmax=3)
                    'halo_id == %d'%halo_id,
    ]   

    cluster_galaxy_data = galaxy_catalog.get_quantities(
                        quantity_cluster_galaxy, 
                        filters=filter_cluster_galaxy, 
                        native_filters=build_query_from_disc(ra_cls, 
                                                             dec_cls, 
                                                             radius_cls)
                                                )
    
    cluster_galaxy_table = Table(cluster_galaxy_data)
    
    cluster_mass.append(halo_mass_cls)
    cluster_richness.append(len(cluster_galaxy_table))
    
#cluster_galaxy_table.write(cluster_catalog_file, 
#                path='/cluster_galaxy_table_'+str(cluster_id), 
#                append=True, overwrite=True)

In [ ]:
#print(cluster_richness,cluster_mass)

In [ ]:
plt.figure(figsize=(8,6))
plt.loglog(cluster_richness, 
            cluster_mass, 
            '.',
            alpha=0.8,
           )

plt.xlabel('Richness $\lambda$')
plt.ylabel('Halo mass [$M_{\odot}$]')
plt.title('0.3<$z$<0.4')
plt.grid(which='both')
plt.rcParams.update({'font.size': 10})
plt.savefig('lambda_m_fig1.png')